<a href="https://colab.research.google.com/github/gauriagarwal18/Project1--Web-Scarping/blob/master/web_scraping_git_topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***SCRAPPING TOP REPOSITORIES FOR TOPICS ON GITHUB ***

#WEB SCRAPING

####-Introduction to web scrapping-: 
Web scraping, web harvesting, or web data extraction is data scraping used for extracting data from websites for drawing conclusions from the data or for performing other analysis.

####-Introduction about GitHub-:
GitHub, Inc. is a provider of Internet hosting for software development and version control using Git. It offers the distributed version control and source code management functionality of Git

link to github: https://github.com

# Project Outline


-we are going to scrape - https://github.com/topics website

-we will get a list of topic and for each topic, we'll get topic title, topic page url,and topic description

-for each topic we will get top 25 repositories in that topic from the topic page.

-for each repository we will mention its name,username, stars,repo url

-for each topic we will create a csv file in the following format-------
```
repo_name,username,stars,repo_url
three.js,mrdoob,77700,https://github.com/mrdoob/three.js
```
-and then we will put all the csv files as an excel sheet in an excel file.


#Installing required libraries and importing them.

###pandas for collecting data
for collecting all the info in the form of a data frame and stroring it in excel file or in csv file as required

### requests library to download web pages
Inspect the website's HTML source and identify the right URLs to download.
Download and save web pages locally using the requests library.
Create a function to automate downloading for different topics/search queries.

###BeautifulSoup to parse and extract information
Parse and explore the structure of downloaded web pages using Beautiful soup.
Use the right properties and methods to extract the required information.
Create functions to extract from the page into lists and dictionaries.
(Optional) Use a REST API to acquire additional information if required.
create the csv file


In [ ]:

import pandas as pd
!pip install requests
import requests
!pip install beautifulsoup4
from bs4 import BeautifulSoup


#Function: scrape_topics
for getting info about the topics in git i will make a function:

####Parameters-

*it will take the url of page and extract the info.

####Use

*it parse the html contents of web page and 

*extract topic name,url,dsescription

*combine the information in a single dataframe


####Return

*dataframe containing the info about the topic

In [ ]:
def scrape_topics(url='https://github.com/topics'):

  #extractting the html code of the website which we want to extract
  response=requests.get(url)
  page_content=response.text

  #parsing the above requested website so we can draw useful info from it
  soup=BeautifulSoup(page_content,'html.parser')
  selection_class="f3 lh-condensed mb-0 mt-1 Link--primary"
  topic_title_tags=soup.find_all('p',{'class':selection_class})    

  #topic discription tags
  selection_class_des="f5 color-fg-muted mb-0 mt-1"
  topic_description_tags=soup.find_all('p',{'class':selection_class_des}) 

  #topics link tags
  selection_class_link="no-underline flex-1 d-flex flex-column"
  topic_url_tags=soup.find_all('a',{'class':selection_class_link})

  #getting topics titles ,topic tags ,topic urls
  base_url="https://github.com"
  topic_titles=[tag.text for tag in topic_title_tags]
  topic_description=[(tag.text).strip() for tag in topic_description_tags]
  topic_url=[base_url+tag['href'] for tag in topic_url_tags]
  topics_df=pd.DataFrame({'titles':topic_titles,'url':topic_url,'description':topic_description})

  return topics_df

# for top repositores of each project

Created two functions for the end-to-end process of downloading, parsing, and saving CSVs.

##I have made two functions:
###1st function: scrape_topics_repo

####Parameter

*it will take the topic url as a paremeter

####Use

*Collect the info, scrape the page of that particular topic

*extract the  top repositories names,usernames,stars,repo url

*save all the information in a dataframe.

####Return

*return a data frame which have all the information about top repositries of a project.


In [ ]:
def scrape_topics_repos(url):

  #use this base url for getting complete urls of each repositore
  base_url="https://github.com"

  #reading the website in text format
  response0=requests.get(url)
  page_content0=response0.text
  
  #parsing the html content of website
  soup0=BeautifulSoup(page_content0,'html.parser')
  class_parent="f3 color-fg-muted text-normal lh-condensed"

  #finding the parent tag which contains user name,repo name and repo url
  repo_parent_tags=soup0.find_all('h3',{'class':class_parent})
  
  #iterating one by one for each repository
  repo_user=[]
  repo_name=[]
  repo_url=[]
  for repo in repo_parent_tags:
    a_tag=repo.find_all('a')
    repo_user.append(a_tag[0].text.strip())
    repo_name.append(a_tag[1].text.strip())
    repo_url.append(base_url+a_tag[1]['href'])
  

  #for getting stars of each repository
  class_stars="Counter js-social-count"
  star_tags=soup0.find_all("span",{'class':class_stars})

  repo_stars=[]
  for tag in star_tags:
    if(tag.text[-1]=='k'):
      repo_stars.append(float(tag.text[:-1])*1000)
    else:
      repo_stars.append(float(tag.text))

  #for making dictionary of info of all repositories
  repo_dict={"user":repo_user,"name":repo_name,"url":repo_url,"stars":repo_stars}

  #collecting all info in a dataframe
  repo_df=pd.DataFrame(repo_dict)
  return repo_df





###2nd function: make_file

####Parameter-

*It will take url of github topic page link as a parameter

####Use
*it will traverse throughth the url column of topics data frame

*one by one call the function and get the data of top repos

*save the data as a separate sheet of excel file with name as its topic name

####Return
*none (it will not return anything)

In [ ]:
def make_file(url):

  topics_df=scrape_topics(url)             #get the dataframe which have info about the topic
  topics_df.to_excel('github_topics_top_repos.xlsx',sheet_name="git_topics")
  for name,url in zip(topics_df['titles'],topics_df['url']):
    repos=scrape_topics_repos(url)
    with pd.ExcelWriter('github_topics_top_repos.xlsx',mode='a') as writer:  # doctest: +SKIP
      repos.to_excel(writer, sheet_name=name)


In [ ]:
#calling the function for creating the excel file
def main():
  url='https://github.com/topics' 
  make_file(url)                 #make as excel file with sheets each having info about top repositories of the repective topics


In [ ]:
#calling our main function where link to our target page is saved.

if __name__ == "__main__":
  main()

###Explaination of how all the functions are interacting with each other----

*main function will be called at first.

*main function consits the url of target page and it will call the make_file function to generate the excel file.

*make_file will then call scrape_topics function to get the datarame with information about each topic.

*Then it will iterate through the dataset and extract info about the top repositories of each project by calling scrape_topics_repo function.

*and then the excel file is created with the required information
